In [12]:
import polars as pl
import pandas as pd
import datetime
import numpy as np
data = pl.read_csv("../../dataset/train.csv",try_parse_dates=True)
df_data = data.with_columns(pl.col("datetime").dt.date().alias("date").cast(pl.Date))
data_pred = data.filter(pl.col("datetime")>=datetime.datetime(2021, 9, 30, 0, 0)).to_pandas()
data =  data.filter(pl.col("datetime")<datetime.datetime(2021, 9, 30, 0, 0))

In [13]:
def previous_value(df_data,day=7):
    last_week = df_data[["county","is_business","product_type","prediction_datetime","is_consumption","target"]]
    last_week["prediction_datetime"] = last_week["prediction_datetime"]+pd.Timedelta('7 day')
    return last_week

current_data = data[["county","is_business","product_type","datetime","is_consumption","target"]].to_pandas()
current_data = current_data.rename(columns={"datetime":"prediction_datetime"})
time_machine = previous_value(current_data).reset_index(drop=True)
time_machine["is_business"] = time_machine["is_business"].astype(bool)
time_machine["is_consumption"] = time_machine["is_consumption"].astype(bool)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.000,0,2021-09-30 00:00:00,29,84912,0
1,0,0,1,127.450,1,2021-09-30 00:00:00,29,84913,0
2,0,0,2,0.000,0,2021-09-30 00:00:00,29,84914,1
3,0,0,2,24.407,1,2021-09-30 00:00:00,29,84915,1
4,0,0,3,0.035,0,2021-09-30 00:00:00,29,84916,2
...,...,...,...,...,...,...,...,...,...
1933435,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64
1933436,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59
1933437,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59
1933438,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60


In [5]:
time_machine

,county,is_business,product_type,prediction_datetime,is_consumption,target
0,0,False,1,2021-09-08 00:00:00,False,0.713
1,0,False,1,2021-09-08 00:00:00,True,96.590
2,0,False,2,2021-09-08 00:00:00,False,0.000
3,0,False,2,2021-09-08 00:00:00,True,17.314
4,0,False,3,2021-09-08 00:00:00,False,2.904
...,...,...,...,...,...,...
84907,15,False,3,2021-10-06 23:00:00,True,81.081
84908,15,True,1,2021-10-06 23:00:00,False,0.100
84909,15,True,1,2021-10-06 23:00:00,True,25.612
84910,15,True,3,2021-10-06 23:00:00,False,0.000


In [3]:
import enefit
env = enefit.make_env()
iter_test = env.iter_test()

In [4]:
counter = 0
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    merged = test.merge(time_machine,how="left",on=["county","is_business","product_type","prediction_datetime","is_consumption"])
    preds = {rid:predd if np.isfinite(predd) else 0  for rid,predd in zip(merged.row_id,merged.target)}
    sample_prediction['target'] = sample_prediction.row_id.apply(lambda x: preds[x] if x in preds else 0)
    if revealed_targets is not None:
        revealed_targets.target = revealed_targets.target.fillna(0)
        time_machine = pd.concat([time_machine,previous_value(revealed_targets.rename(columns={"datetime":"prediction_datetime"}))])
        time_machine = time_machine[time_machine.prediction_datetime>=(test.prediction_datetime[0]-pd.Timedelta('7 day'))].reset_index(drop=True)
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


/tmp/ipykernel_32/3978881576.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_week["prediction_datetime"] = last_week["prediction_datetime"]+pd.Timedelta('7 day')
/tmp/ipykernel_32/3978881576.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_week["prediction_datetime"] = last_week["prediction_datetime"]+pd.Timedelta('7 day')
/tmp/ipykernel_32/3978881576.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,row_id,target
0,2015232,2.045
1,2015233,426.177
2,2015234,0.000
3,2015235,3.862
4,2015236,15.618
...,...,...
3115,2018347,415.530
3116,2018348,0.000
3117,2018349,31.286
3118,2018350,0.000
